In [ ]:
import numpy as np
import cv2
import imageio
from PIL import Image
import matplotlib.pyplot as plt
import time
import math
import glob
import matplotlib.colors as colors
from skimage import color, io, exposure
from scipy.ndimage import morphology as morph
from skimage.morphology import disk
from skimage.transform import resize
from skimage import filters
#%matplotlib inline

In [ ]:
def calcErrorSurface(panorama, curr_img, overlap, channel):
    A = panorama[:, -overlap-1:, channel]
    B = curr_img[:, 0:overlap+1, channel]
    return np.square(A-B)

def calcSeam(e):
    E = np.zeros(e.shape);
    E[0, :] = e[0, :];
    for h in range(1, e.shape[0]):
        for w in range(0, e.shape[1]):
            if w == 0:
                cost = min(E[h-1, w], E[h-1, w+1]);
            elif w == e.shape[1]-1:
                cost = min(E[h-1, w-1], E[h-1, w]);
            else:
                cost = min(E[h-1, w-1], E[h-1, w], E[h-1, w+1]);
            E[h, w] = e[h, w] + cost;
    return E

def calcPanorama(images_dir, shift):
    start = time.time()
    # read panorama source images
    files = glob.glob(images_dir + 'in-*.*g');
    files = sorted(files)
    print(len(files))
    
    image_files = [np.array(Image.open(files[i])) for i in range(len(files))];
    
    images_temp = [ image_files[i].astype('float64') for i in range(len(image_files))];
    
    if image_files[0].ndim == 2 or image_files[0].shape[2] == 1:
        images_temp = [ cv2.resize(cv2.cvtColor(image_files[i], cv2.COLOR_GRAY2RGB), (200, 300)).astype('float64') for i in range(len(image_files))];
    
    panorama = images_temp[0];
    for i in range(1, len(images_temp)):
        curr_img = images_temp[i];
        
        channel = np.argmax([np.var(curr_img[:,:,0]), np.var(curr_img[:,:,1]), np.var(curr_img[:,:,2])]);
        
        overlap = curr_img.shape[1] - shift[i-1];
        e = calcErrorSurface(panorama, curr_img, overlap, channel);
        print(e)

In [ ]:
calcPanorama('/home/additya/Downloads/images/images/3/', [55]*11);